In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import copy

In [2]:
file_t = open('jester-data-1.csv').readlines()

In [3]:
mp = pd.read_csv('jester-data-1.csv', header=None).values

In [4]:
mp

array([[ 74.  ,  -7.82,   8.79, ...,  99.  ,  99.  ,  99.  ],
       [100.  ,   4.08,  -0.29, ...,   0.34,  -4.32,   1.07],
       [ 49.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       ...,
       [ 39.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       [ 37.  ,  99.  ,  99.  , ...,  99.  ,  99.  ,  99.  ],
       [ 72.  ,   2.43,   2.67, ...,  99.  ,  99.  ,  99.  ]])

In [6]:
def get_matrix():
    return pd.read_csv('jester-data-1.csv', header=None, na_values=99).iloc[:,1:].values

In [249]:
class JokerObject:
    def __init__(self, matrix):
        self.matrix = matrix
        self.item_means = np.nanmean(matrix, 0)
        self.user_means = np.nanmean(matrix, 1)
    def get_matrix(self):
        return self.matrix
    def get_items_m(self):
        return self.item_means
    def get_item_m(self, key):
        return self.item_means[key]
    def get_users_m(self):
        return self.user_means
    def get_user_m(self, key):
        return self.user_means[key]
    def get_item(self, row, col):
        return self.matrix[row][col]
    def set_object_nan(self, row, col):
        self.matrix[row][col] = np.nan
    def get_pearson_matrix(self):
        distances = np.zeros((len(self.item_means),len(self.item_means)))
        transposes = self.matrix.transpose()
        t0 = dt.datetime.now()
        for i, item_1 in enumerate(transposes):
            if i % 1 == 0:
                print(i)
                print("time: ", dt.datetime.now() - t0)
            for j, item_2 in enumerate(transposes):
                if(i>j):
                    continue
                if i != j:
                    top = np.sum((a-self.item_means[i])*(b-self.item_means[j]) for a,b in zip(item_1,item_2) if np.isnan(a)== False and np.isnan(b) == False)
                    bot = np.sqrt(np.sum(np.square(a-self.item_means[i]) for index, a in enumerate(item_1) if np.isnan(a)== False and np.isnan(item_2[index]) == False)*np.sum(np.square(b-self.item_means[j]) for index, b in enumerate(item_2) if np.isnan(b)== False and np.isnan(item_1[index]) == False))
                    distances[i,j] = top/bot 
                    distances[j,i] = distances[i,j]
        return distances
                    

In [250]:
t = np.array([1,3,9])
t2 = np.array([5,6,2])
t3 = np.array([3,2,8])
t4 = np.array([t,t2,t3])

In [251]:
t4 = JokerObject(get_matrix())

In [252]:
#t4.get_matrix().transpose()

In [253]:
#m = JokerObject(t4)

In [254]:
# Got distance matrix
t0 = dt.datetime.now()
dist = t4.get_pearson_matrix()
print("time: ", dt.datetime.now() - t0)

0
time:  0:00:00.000073
1
time:  0:00:35.194998
2
time:  0:01:11.520558
3
time:  0:01:46.592903
4
time:  0:02:20.003958
5
time:  0:02:59.880621
6
time:  0:03:36.265339
7
time:  0:04:14.722614
8
time:  0:04:52.577532
9
time:  0:05:24.304114
10
time:  0:05:58.529913
11
time:  0:06:33.943478
12
time:  0:07:08.957152
13
time:  0:07:44.828171
14
time:  0:08:18.939866
15
time:  0:08:54.323684
16
time:  0:09:28.949989
17
time:  0:10:03.339798
18
time:  0:10:37.074643
19
time:  0:11:10.220716
20
time:  0:11:42.868595
21
time:  0:12:14.932576
22
time:  0:12:44.742466
23
time:  0:13:13.536172
24
time:  0:13:39.948080
25
time:  0:14:08.316974
26
time:  0:14:38.223376
27
time:  0:15:08.058685
28
time:  0:15:36.878455
29
time:  0:16:05.797303
30
time:  0:16:31.118612
31
time:  0:16:58.655709
32
time:  0:17:26.163850
33
time:  0:17:49.688426
34
time:  0:18:14.949165
35
time:  0:18:40.979413
36
time:  0:19:06.630555
37
time:  0:19:28.736277
38
time:  0:19:52.578892
39
time:  0:20:16.314728
40
time:  

In [287]:
'''
    Distance matrix 100 x 100 by items
    coords is (user, item) to guess
'''
def get_avg_weighted_sum(distance_matrix, data_matrix, coords):
    if not np.isnan(data_matrix.get_item(coords[0],coords[1])):
        data_matrix = copy.deepcopy(data_matrix)
        data_matrix.set_object_nan(coords[0],coords[1])
    return data_matrix.get_item_m(coords[1]) + (1/distance_matrix.sum(axis=0)[coords[1]])*np.sum(distance_matrix[coords[1]][c_prime]*(data_matrix.get_item(coords[0], c_prime)-data_matrix.get_item_m(c_prime))for c_prime in range(distance_matrix.shape[0]) if not np.isnan(data_matrix.get_item(coords[0], c_prime)))


In [288]:
file = open("distance_items.txt",'w')
for row in dist:
    row = [str(x) for x in row]
    file.write(",".join(row) + "\n")
file.close()

In [289]:
get_avg_weighted_sum(dist,t4,(0,1))

-3.2671252495585623

In [281]:
t4.get_matrix()

array([[-7.82,  8.79, -9.66, ...,   nan,   nan,   nan],
       [ 4.08, -0.29,  6.36, ...,  0.34, -4.32,  1.07],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [ 2.43,  2.67, -3.98, ...,   nan,   nan,   nan]])

In [285]:
t = np.array([[1,2],[3,4]])
t

array([[1, 2],
       [3, 4]])

In [286]:
t.sum()

10

In [77]:
def get_average_ranking(data_matrix, coords):
    if not np.isnan(data_matrix.get_item(coords[0],coords[1])):
        data_matrix = copy.deepcopy(data_matrix)
        data_matrix.set_object_nan(coords[0],coords[1])
    utilities = [data_matrix.get_item(c,coords[1])for c in range(data_matrix.get_matrix().shape[0]) if not np.isnan(data_matrix.get_item(c, coords[1]))]
    return (1/len(utilities))*sum(utilities)

In [78]:
get_average_ranking(t4, (0,53))

2.372631578947368

In [292]:
def get_adjusted_weighted_Nnn_sum(distance_matrix, data_matrix, coords, N = 3):
    if not np.isnan(data_matrix.get_item(coords[0],coords[1])):
        data_matrix = copy.deepcopy(data_matrix)
        data_matrix.set_object_nan(coords[0],coords[1])
    return data_matrix.get_item_m(coords[1]) + (1/distance_matrix.sum(axis=0)[coords[1]])*np.sum(distance_matrix[coords[1]][c_prime]*(data_matrix.get_item(coords[0], c_prime)-data_matrix.get_item_m(c_prime))for c_prime in dist[coords[1],].argsort()[:N] if not np.isnan(data_matrix.get_item(coords[0], c_prime)))

In [294]:
get_adjusted_weighted_Nnn_sum(dist, t4, (0,2), 4)

0.22701042946740263

In [56]:
tmp = dist[np.arange(dist.shape[0]) != 1][:,1]

In [296]:
t4.get_matrix()

array([[-7.82,  8.79, -9.66, ...,   nan,   nan,   nan],
       [ 4.08, -0.29,  6.36, ...,  0.34, -4.32,  1.07],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       ...,
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [  nan,   nan,   nan, ...,   nan,   nan,   nan],
       [ 2.43,  2.67, -3.98, ...,   nan,   nan,   nan]])

In [57]:
np.abs(tmp).argsort()

array([2, 1, 3, 0])

In [58]:
np.delete(dist[0,],0).argsort()[:3]

array([0, 1, 3])

In [59]:
t = np.delete(tmp,0)

In [60]:
t.argsort()[:3]

array([2, 0, 1])

In [61]:
dist[np.arange(dist.shape[0]) != 1]

array([[ 0.        , -0.23975997,  0.08422171,  0.39590991,  0.33005565],
       [ 0.08422171, -0.07953065,  0.        ,  0.34000639,  0.33146463],
       [ 0.39590991,  0.04503484,  0.34000639,  0.        ,  0.44410961],
       [ 0.33005565, -0.11300586,  0.33146463,  0.44410961,  0.        ]])

In [73]:
dist.shape

(100, 100)

In [74]:
dist[0]

array([ 0.00000000e+00,  4.29376816e-01,  4.97969880e-01,  5.31639699e-01,
        3.54819055e-01,  1.62217803e-01,  4.45399817e-01,  4.64562150e-01,
        4.62404706e-01,  3.76828809e-01,  5.17247003e-01,  3.40088394e-01,
        1.52691477e-01,  2.65879811e-01,  1.29786999e-01,  7.09107331e-02,
        2.22409331e-01,  3.77401155e-02,  2.15610835e-01,  2.06294832e-01,
        2.50638519e-01,  2.31253304e-01,  3.05092245e-01,  2.91004111e-01,
        4.75663833e-01,  3.11785952e-01,  1.90120139e-02,  1.93114225e-01,
        2.32691677e-01,  3.03513358e-01,  2.40951409e-01,  1.61943335e-01,
        3.83485923e-01,  1.21628885e-01,  1.71521336e-01,  9.37446322e-02,
        5.03658407e-01,  4.72663852e-01,  1.41568087e-01,  6.63322286e-01,
        5.69550525e-01,  5.85647194e-03,  3.86901115e-01,  9.47271586e-02,
        2.11145525e-01,  2.87117453e-01,  1.16695293e-01,  3.48942520e-01,
        1.31710376e-01, -1.77699740e-03,  3.86127902e-01,  2.65640350e-01,
       -3.21791188e-02,  

In [24]:
t = np.matrix(dist)

matrix([[-0.23975997,  0.        , -0.07953065,  0.04503484, -0.11300586,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.

In [19]:
dist

array([[ 0.        , -0.23975997,  0.08422171, ...,  0.        ,
         0.        ,  0.        ],
       [-0.23975997,  0.        , -0.07953065, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.08422171, -0.07953065,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [29]:
dist.transpose()

array([[ 0.        , -0.23975997,  0.08422171, ...,  0.        ,
         0.        ,  0.        ],
       [-0.23975997,  0.        , -0.07953065, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.08422171, -0.07953065,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [121]:
t = np.matrix([[np.nan,2,4],[4,5,6]])
t

matrix([[nan,  2.,  4.],
        [ 4.,  5.,  6.]])

In [122]:
range(t.shape[1])

range(0, 3)

In [124]:
t[:,0]

matrix([[nan],
        [ 4.]])

In [111]:
    def get_pearson_matrix(self):
        distances = np.zeros((len(self.item_means),len(self.item_means)))
        transposes = np.matrix(self.matrix).transpose()
        for item_1 in range(transposes.shape[1]):
            for item_2 in enumerate(transposes.shape[1]):
                if(i>j):
                    continue
                if i != j:
                    
                    top = np.sum((a-self.item_means[i])*(b-self.item_means[j]) for a,b in zip(transposes[:,item_1],transposes[:,item_2]) if np.isnan(a)== False and np.isnan(b) == False)
                    bot = np.sqrt(np.sum(np.square(a-self.item_means[i]) for index, a in enumerate(transposes[:,item_1]) if np.isnan(a)== False and np.isnan(item_2[index]) == False)*np.sum(np.square(b-self.item_means[j]) for index, b in enumerate(transposes[:,item_2]) if np.isnan(b)== False and np.isnan(item_1[index]) == False))
                    distances[i,j] = top/bot 
                    distances[j,i] = distances[i,j]
        return distances

In [128]:
for i in t[:,0]:
    print(np.isnan(i) == True)

[[ True]]
[[False]]
